# Analysis of intrinsic results: GeoDanmark

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle
import json
import pandas as pd
import seaborn as sns
import plotly.express as px
import h3

from src import evaluation_functions as eval_func

%run ../settings/yaml_variables.py
%run ../settings/df_styler.py
%run ../settings/plotting.py
%run ../settings/load_refdata.py

In [ ]:
# Read intrinsic grid results
with open(
    f"../results/REFERENCE/{study_area}/data/grid_results_intrinsic.pickle", "rb"
) as fp:
    ref_intrinsic_grid = pickle.load(fp)

# Import intrinsic results
ref_intrinsic_file = open(
    f"../results/REFERENCE/{study_area}/data/intrinsic_analysis.json"
)

ref_intrinsic_results = json.load(ref_intrinsic_file)

# Import summary dataframe
summarize_results_df = pd.read_csv(f"../results/REFERENCE/{study_area}/data/intrinsic_summary_results.csv",index_col=0)

summarize_results_df.style.pipe(format_ref_style)

## Municipal Analysis

In [ ]:
# Read muni data
muni = gpd.read_file("../data/municipalities.gpkg")
muni = muni[['navn','kommunekode','geometry']]
assert muni.crs == study_crs

In [ ]:
def index_by_muni(gdf, muni, muni_cols=['navn','kommunekode'], assert_len=True, drop_dupli=False, dupli_id=None):

    join = gdf.sjoin(muni, predicate="intersects", how="left")
    join.drop('index_right',axis=1,inplace=True)

    if len(join[join[muni_cols[0]].isna()]) > 0:

        non_joined = join[join[muni_cols[0]].isna()].copy()

        non_joined.drop(muni_cols,axis=1,inplace=True)

        join_2 = non_joined.sjoin_nearest(muni, how="left")

        if 'index_right' in join_2.columns:
            join_2.drop('index_right',axis=1,inplace=True)

        join.dropna(subset=muni_cols[0], inplace=True)

        if drop_dupli:
            join_2.drop_duplicates(subset=dupli_id, keep="first",inplace=True)
            join.drop_duplicates(subset=dupli_id, keep="first",inplace=True)

        if assert_len:
            
            assert len(join_2) + len(join) == len(gdf)

        final_join = pd.concat([join,join_2])
    
    else:

        if drop_dupli:
            join.drop_duplicates(subset=dupli_id, keep="first",inplace=True)

        final_join = join

    if assert_len:
        
        assert len(final_join) == len(gdf)

    assert len(final_join[final_join[muni_cols[0]].isna()]) == 0

    return final_join

**Assign municipal ID/name to all grid cells**

In [ ]:
# Assign municipal name and id to each hex cell based on centroid overlap
ref_intrinsic_grid.dropna(subset='count_ref_edges',inplace=True)

grid_centroids = ref_intrinsic_grid[['geometry','grid_id','count_ref_edges']].copy()
grid_centroids['geometry'] = grid_centroids.geometry.centroid

joined_int_grid = index_by_muni(grid_centroids, muni)

int_grid = ref_intrinsic_grid.merge(joined_int_grid[['grid_id','navn','kommunekode']],left_on='grid_id',right_on='grid_id')
assert len(int_grid) == len(ref_intrinsic_grid)

**Index nodes and edges by municipality**

In [ ]:
# Index nodes and edges by municipality
muni_edges = index_by_muni(ref_edges_simplified, muni, assert_len=False)

muni_nodes = index_by_muni(ref_nodes_simplified, muni, assert_len=True, drop_dupli=True, dupli_id='nodeID')
assert len(muni_nodes) == len(muni_nodes.nodeID.unique())

# Recompute length - for GeoDK, infra length is equal to geom length
muni_edges['infrastructure_length'] = muni_edges.geometry.length

# Index dangling nodes by muni
muni_dang = index_by_muni(ref_dangling, muni, assert_len=True, drop_dupli=True, dupli_id='nodeID')
assert len(muni_dang) == len(muni_dang.nodeID.unique())

# Group by muni
grouped_edges = muni_edges.groupby("navn") 
grouped_nodes = muni_nodes.groupby("navn")
grouped_dangling = muni_dang.groupby("navn")

# Turn into dataframe
muni_infra = grouped_edges['infrastructure_length'].sum().to_frame()

muni_node_count = grouped_nodes.size().to_frame('node_count') 

muni_dang_count = grouped_dangling.size().to_frame('dangling_node_count') 

muni_network_counts = pd.merge(pd.merge(muni_infra, muni_node_count,left_index=True, right_index=True),muni_dang_count,left_index=True, right_index=True)

assert len(muni_network_counts) == 98


**Plot length of bicycle infrastructure, bicycle infrastructure density and infrastructure per 1000 people**

In [ ]:
# Plot km of bicycle infra per muni
muni_network_counts["infra_km"] = round(
    muni_network_counts.infrastructure_length / 1000, 3
)

fig, ax = plt.subplots(figsize=(20, 20))
sns.barplot(
    muni_network_counts.reset_index().sort_values("infra_km"),
    x="navn",
    y="infra_km",
    ax=ax,
    color="red",
)
plt.xticks(rotation=45, ha="right")
plt.xlabel("")
plt.ylabel("KM")
plt.title("Bicycle Infrastructure")

# Plot infra density per muni
muni["area_sqkm"] = round(muni.area / 1000000, 2)

muni_network_counts = muni_network_counts.merge(
    muni[["navn", "area_sqkm"]], left_index=True, right_on="navn"
)

muni_network_counts["infra_dens"] = round(
    muni_network_counts.infra_km / muni_network_counts.area_sqkm, 3
)

fig, ax = plt.subplots(figsize=(20, 20))
sns.barplot(
    muni_network_counts.reset_index().sort_values("infra_dens"),
    x="navn",
    y="infra_dens",
    ax=ax,
    color="red",
)
plt.xticks(rotation=45, ha="right")
plt.xlabel("")
plt.ylabel("KM/SQKM")
plt.title("Bicycle Infrastructure Density")

# Plot infra per pop
muni_pop = pd.read_csv("../data/muni_pop.csv", encoding="ISO-8859-1", header=None)
muni_pop.rename({1: "navn", 2: "pop"}, inplace=True, axis=1)

muni_network_counts = muni_network_counts.merge(
    muni_pop[["navn", "pop"]], left_on="navn", right_on="navn"
)
assert len(muni_network_counts) == 98

muni_network_counts["infra_pop"] = muni_network_counts.infra_km / (
    muni_network_counts["pop"] / 1000
)

fig, ax = plt.subplots(figsize=(20, 20))
sns.barplot(
    muni_network_counts.reset_index().sort_values("infra_pop"),
    x="navn",
    y="infra_pop",
    ax=ax,
    color="red",
)
plt.xticks(rotation=45, ha="right")
plt.xlabel("")
plt.ylabel("KM/1000 people")
plt.title("Bicycle Infrastructure Density: Per 1.000 People");


**Index over and undershoots by municipality**

In [ ]:
# Join over and undershoot to muni
overshoot_edge_ids = pd.read_csv(ref_results_data_fp+"overshoot_edges_3.csv")
undershoot_node_ids = pd.read_csv(ref_results_data_fp+"undershoot_nodes_3.csv")

undershoots = ref_nodes_simplified[ref_nodes_simplified.nodeID.isin(undershoot_node_ids.node_id)]
assert len(undershoot_node_ids) == len(undershoots)

overshoots = ref_edges_simplified[ref_edges_simplified.edge_id.isin(overshoot_edge_ids.edge_id)]
assert len(overshoot_edge_ids) == len(overshoots)

# Join to muni
over_muni = overshoots[['edge_id','geometry']].sjoin(muni,how='left',predicate='intersects')

under_muni = undershoots[['nodeID','geometry']].sjoin(muni,how='left', predicate='intersects')

assert len(over_muni) == len(overshoots)
assert len(under_muni) == len(undershoots)

# Group by muni and count
over_df = over_muni.groupby('navn').size().to_frame('overshoots')

under_df = under_muni.groupby('navn').size().to_frame('undershoots')

# Combine with muni_network
muni_network_counts = pd.merge(muni_network_counts, over_df,left_on='navn', right_on='navn', how='left')
assert len(muni_network_counts) == 98

muni_network_counts = pd.merge(muni_network_counts, under_df,left_on='navn', right_on='navn', how='left')
assert len(muni_network_counts) == 98

muni_network_counts['over_under'] = muni_network_counts.overshoots + muni_network_counts.undershoots
muni_network_counts['over_under'].fillna(0,inplace=True)

**Join over and undershoots to grid cells**

In [ ]:
# Join over and undershoots to grid ids
overshoots_grid = overshoots[['edge_id','geometry']].sjoin(int_grid,how='left').drop_duplicates(subset='edge_id',keep="first")[['grid_id']].reset_index(drop=True)
undershoots_grid = undershoots[['nodeID','geometry']].sjoin(int_grid,how='left').drop_duplicates(subset='nodeID',keep="first")[['grid_id']].reset_index(drop=True)

over_grid_grouped = overshoots_grid.groupby('grid_id').size().to_frame('overshoots_count')
under_grid_grouped = undershoots_grid.groupby('grid_id').size().to_frame('undershoots_count')

int_grid_org_len = len(int_grid)

int_grid = int_grid.merge(over_grid_grouped, left_on='grid_id',right_on='grid_id',how='left')
assert len(int_grid) == int_grid_org_len

int_grid = int_grid.merge(under_grid_grouped, left_on='grid_id',right_on='grid_id',how='left')
assert len(int_grid) == int_grid_org_len

int_grid['overshoots_count'].fillna(0,inplace=True)
int_grid['undershoots_count'].fillna(0,inplace=True)


**Join component gaps to municipality**

In [ ]:
comp_gaps = gpd.read_file(ref_results_data_fp+"component_gaps_centroids_10.gpkg")
comp_gaps.head()

# Join to muni
gaps_muni = comp_gaps.sjoin(muni,how='left',predicate='intersects')

assert len(gaps_muni) == len(comp_gaps)

# Group by muni and count
gaps_df = gaps_muni.groupby('navn').size().to_frame('component_gaps')

# Combine with muni_network
muni_network_counts = pd.merge(muni_network_counts, gaps_df,left_on='navn', right_on='navn', how='left')
assert len(muni_network_counts) == 98

muni_network_counts['component_gaps'] = muni_network_counts.component_gaps.fillna(0).astype(int)

**Join component gaps to grid cells**

In [ ]:
comp_gaps['gap_id'] = comp_gaps.index

gaps_grid = comp_gaps[['gap_id','geometry']].sjoin(int_grid,how='left').drop_duplicates(subset='gap_id',keep="first")[['grid_id']].reset_index(drop=True)
assert len(gaps_grid) == len(comp_gaps)

gaps_grouped = gaps_grid.groupby('grid_id').size().to_frame('component_gaps')

int_grid_org_len = len(int_grid)

int_grid = int_grid.merge(gaps_grouped, left_on='grid_id',right_on='grid_id',how='left')
assert len(int_grid) == int_grid_org_len

int_grid['component_gaps'] = int_grid['component_gaps'].fillna(0).astype(int)

**Correlation between infrastructure length and topology errors**

In [ ]:
# TODO: Add label for high/low density and add as color?
fig = px.scatter(
    muni_network_counts, 
    x='infra_km', 
    y='component_gaps',
    title='Correlation between infrastructure length and component gaps', 
    hover_data=['navn'],
    labels={
        "infra_km": "Length of bicycle infrastructure (KM)",
        "component_gaps": "Gaps between disconnected components",
        "navn": "Municipality"
        })

fig.update_layout(
    font=dict(
        size=10,
        color="RebeccaPurple"
    )
)
fig.show()


# TODO: Add label for high/low density and add as color?
fig = px.scatter(
    muni_network_counts, 
    x='infra_km', 
    y='over_under',
    title='Correlation between infrastructure length and topology errors', 
    hover_data=['navn'],
    labels={
        "infra_km": "Length of bicycle infrastructure (KM)",
        "over_under": "Over and undershoots",
        "navn": "Municipality"
        })

fig.update_layout(
    font=dict(
        size=10,
        color="RebeccaPurple"
    )
)
fig.show()


**Index components by municipality**

In [ ]:
muni_grid = int_grid.groupby('navn')

muni_components = {}

for name, group in muni_grid:
    nested_list = group.component_ids_ref.to_list()
    flat_list = list(set([item for sublist in nested_list for item in sublist]))

    muni_components[name] = len(flat_list)

muni_comp_df = pd.DataFrame.from_dict(muni_components,orient='index',columns=['component_count']).reset_index().rename({'index':'navn'},axis=1)
assert len(muni_comp_df) == 98

muni_network_counts = muni_network_counts.merge(muni_comp_df, left_on='navn', right_on='navn', how='left')
assert len(muni_network_counts) == 98


**Compute municipal components per infrastructure**

In [ ]:
muni_network_counts['comp_per_km'] = round(muni_network_counts.component_count / muni_network_counts.infra_km,2)

In [ ]:
# Plot correlation between infrastructure length and no of componentsß
fig = px.scatter(
    muni_network_counts,
    x="infra_km",
    y="comp_per_km",
    title="Correlation between components per km and infrastructure length",
    hover_data=["navn"],
    color="area_sqkm",
    color_continuous_scale="viridis_r",
    labels={
        "infra_km": "Length of bicycle infrastructure (km)",
        "component_count": "Number of disconnected components",
        "navn": "Municipality",
        "area_sqkm": "Size (sqkm)",
        "comp_per_km": "Components per km",
    },
)

fig.update_layout(font=dict(size=12, color="RebeccaPurple"))
fig.show()

# Plot correlation between infrastructure length and no of componentsß
fig = px.scatter(
    muni_network_counts,
    x="area_sqkm",
    y="comp_per_km",
    title="Correlation between components per km and municipality size",
    hover_data=["navn"],
    color="infra_km",
    color_continuous_scale="viridis_r",
    labels={
        "infra_km": "Length of bicycle infrastructure (km)",
        "component_count": "Number of disconnected components",
        "navn": "Municipality",
        "area_sqkm": "Size (sqkm)",
        "comp_per_km": "Components per km",
    },
)

fig.update_layout(font=dict(size=12, color="RebeccaPurple"))
fig.show()

# Plot correlation between infrastructure length and no of componentsß
fig = px.scatter(
    muni_network_counts,
    x="infra_dens",
    y="comp_per_km",
    title="Correlation between components per km and infrastructure density",
    hover_data=["navn"],
    # color="infra_km",
    # color_continuous_scale='viridis_r',
    labels={
        "infra_km": "Length of bicycle infrastructure (km)",
        "infra_dens": "Infrastructure density",
        "component_count": "Number of disconnected components",
        "navn": "Municipality",
        "area_sqkm": "Size (sqkm)",
        "comp_per_km": "Components per km",
    },
)

fig.update_layout(font=dict(size=12, color="RebeccaPurple"))

fig.show()


**Export**

In [ ]:
# Export
assert len(muni_network_counts) == 98
muni_network_counts.to_csv("../results/geodk_quality/muni_network_counts.csv", index=False)

int_grid.to_parquet("../results/geodk_quality/int_grid.parquet")

**Plot scatterplots of correlation between number of components and infrastructure length, area, and density**

In [ ]:
# Plot correlation between infrastructure length and no of componentsß
fig = px.scatter(
    muni_network_counts, 
    x='infra_km', 
    y='component_count',
    title='Correlation between infrastructure length and number of components', 
    hover_data=['navn'],
    color="area_sqkm",
    color_continuous_scale='viridis_r',
    labels={
        "infra_km": "Length of bicycle infrastructure (km)",
        "component_count": "Number of disconnected components",
        "navn": "Municipality",
        "area_sqkm": "Size (sqkm)",
        })

fig.update_layout(
    font=dict(
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()

# Plot correlation between municipality size and no of componentsß
fig = px.scatter(
    muni_network_counts, 
    x='area_sqkm', 
    y='component_count',
    color="infra_km",
    title='Correlation between municipality size and number of components', 
    hover_data=['navn'],
    color_continuous_scale='viridis_r',
    labels={
        "area_sqkm": "Size (sqkm)",
        "infra_km": "Length (km)",
        "component_count": "Number of disconnected components",
        "navn": "Municipality"
        })

fig.update_layout(
    font=dict(
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()

# Plot correlation between density and number of components
fig = px.scatter(
    muni_network_counts, 
    x='infra_dens', 
    y='component_count',
    #color="infra_km",
    title='Correlation between infrastructure density and number of components', 
    hover_data=['navn'],
    #color_continuous_scale='viridis_r',
    labels={
        # "area_sqkm": "Size (sqkm)",
        # "infra_km": "Length (km)",
        "infra_dens": "Infrastructure density",
        "component_count": "Number of disconnected components",
        "navn": "Municipality"
        })

fig.update_layout(
    font=dict(
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()


**Plot all over and undershoots**

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))

int_grid.plot(ax=ax,alpha=0.5,color='grey')
overshoots_cent = overshoots.copy()
overshoots_cent['geometry'] = overshoots.centroid
overshoots_cent.plot(ax=ax,color='red')
undershoots.plot(ax=ax,color='blue');

## Spatial patterns in topology errors

In [ ]:
# from pysal.explore import esda
# from pysal.lib import weights
# from splot.esda import lisa_cluster

**Compute local network length**

In [ ]:
hexid = int_grid.loc[0,'hex_id']

h3_res = h3.h3_get_resolution(hexid)
print("The hexagons are at resolution:", h3_res)

k = eval_func.radius_to_k_ring(2, h3_res)

int_grid['k_rings'] = int_grid.apply(lambda x: h3.k_ring(x.hex_id, k),axis=1)

int_grid['area_sqkm'] = int_grid.area / 1000000

int_grid['local_length'] = int_grid.apply(lambda x: eval_func.get_local_network_length(int_grid, 'ref_edge_density', 'area_sqkm', x.k_rings),axis=1)

**Correlation between infrastructure density and topology errors:**

In [ ]:
fig = px.scatter(
    int_grid, 
    x='ref_edge_density', 
    y='undershoots_count',
    title='Correlation between infrastructure density and number of topology errors', 
    hover_data=['navn'],
    #color="navn",
    #color_continuous_scale='viridis_r',
    labels={
        "ref_edge_density": "Infrastructure density (m/sqkm)",
        "undershoots_count": "Number of undershoots",
        #"navn": "Municipality",
        #"area_sqkm": "Size (sqkm)",
        })

fig.update_layout(
    font=dict(
        size=12,
        color="RebeccaPurple"
    )
)
fig.show()

**Compute spatial weights**

In [ ]:
# cents = int_grid.centroid

# # Extract coordinates into an array
# pts = pd.DataFrame(
#     {"X": cents.x, "Y": cents.y}
# ).values

# w = weights.distance.DistanceBand.from_array(
#     pts, 1000, binary=False
# )

# # w = weights.distance.KNN.from_array(
# #     pts, k=12
# # )

# # row standardize
# w.transform = "R"

# #sns.histplot(w.cardinalities, bins=10, kde=True);

**Global Spatial Autocorrelation with Moran's I**

In [ ]:
# error_types = ['undershoots','overshoots','gaps']
# error_cols = ['undershoots_count','overshoots_count','component_gaps']

# for e, c in zip(error_types,error_cols):

#     # compute density
#     int_grid[f'{e}_dens'] = int_grid[c] / (int_grid.local_network_length/1000) # ref_edge_density  #edge_dens_lag

#     # compute spatial lag
#     int_grid[f"{e}_lag"] = weights.spatial_lag.lag_spatial(
#         w, int_grid[f"{e}_dens"]
#     )

#     f, ax = plt.subplots(1, figsize=(6, 6))
#     sns.regplot(
#         x=f"{e}_dens",
#         y=f"{e}_lag",
#         ci=None,
#         data=int_grid,
#         line_kws={"color": "r"},
#     )
#     ax.axvline(0, c="k", alpha=0.5)
#     ax.axhline(0, c="k", alpha=0.5)
#     ax.set_title(f"Moran Plot - {e}")
#     plt.show()

#     moran = esda.moran.Moran(int_grid[f"{e}_dens"], w)
#     print(f"With significance {moran.p_sim}, the Moran's I value for {e} errors is {moran.I}" )


**Local Spatial Autocorrelation with Moran's I**

In [ ]:
# # based on https://geographicdata.science/book/notebooks/07_local_autocorrelation.html

# error_types = ['undershoots','overshoots','gaps']
# error_cols = ['undershoots_count','overshoots_count','component_gaps']

# for e, c in zip(error_types,error_cols):

#     lisa = esda.moran.Moran_Local(int_grid[f'{e}_dens'], w)

#     sig = 1 * (lisa.p_sim < 0.05)

#     spots = lisa.q * sig

#     # Mapping from value to name (as a dict)
#     spots_labels = {
#         0: "Non-Significant",
#         1: "HH",
#         2: "LH",
#         3: "LL",
#         4: "HL",
#     }
#     int_grid[f"{e}_q"] = pd.Series(
#         spots,
#         index=int_grid.index
#         ).map(spots_labels)

#     f, axs = plt.subplots(nrows=2, ncols=2, figsize=(20,20))
#     axs = axs.flatten()

#     ax = axs[0]
#     int_grid.assign(
#         Is=lisa.Is
#     ).plot(
#         column="Is",
#         cmap="plasma",
#         scheme="quantiles",
#         k=2,
#         edgecolor="white",
#         linewidth=0.1,
#         alpha=0.75,
#         legend=True,
#         ax=ax,
#     )

#     ax = axs[1]

#     lisa_cluster(lisa, int_grid, p=1, ax=ax)

#     ax = axs[2]
#     labels = pd.Series(
#         1 * (lisa.p_sim < 0.05), 
#         index=int_grid.index  
#     ).map({1: "Significant", 0: "Non-Significant"})
#     int_grid.assign(
#         cl=labels
#     ).plot(
#         column="cl",
#         categorical=True,
#         k=2,
#         cmap="Paired",
#         linewidth=0.1,
#         edgecolor="white",
#         legend=True,
#         ax=ax,
#     )

#     ax = axs[3]
#     lisa_cluster(lisa, int_grid, p=0.05, ax=ax)

#     for i, ax in enumerate(axs.flatten()):
#         ax.set_axis_off()
#         ax.set_title(
#             [
#                 "Local Statistics",
#                 "Scatterplot Quadrant",
#                 "Statistical Significance",
#                 "Moran Cluster Map",
#             ][i],
#             y=0,
#         )

#     f.suptitle(f'Local Spatial Autocorrelation for: {e}', fontsize=16)

#     f.tight_layout()

#     plt.show()

In [ ]:
# int_grid['component_ids_ref'] = int_grid['component_ids_ref'].astype(str)

# int_grid.to_parquet('../results/geodk_quality/int_grid.parquet')

In [ ]:
# save_cols = ['hex_id', 'geometry',
#        'ref_edge_density','navn', 'kommunekode', 'overshoots_count',
#        'undershoots_count', 'component_gaps', 'area_sqkm',
#        'local_length','undershoots_dens', 'undershoots_lag', 'overshoots_dens',
#        'overshoots_lag','undershoots_q','overshoots_q', 'gaps_dens', 'gaps_lag', 'gaps_q']

# int_grid[save_cols].to_file('../data/int_grid_test.gpkg')